In [4]:
%matplotlib inline
import numpy as np
import scipy as sp
import json 
import csv
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [5]:
df=pd.read_json("./data/auctions_detailed.json")
df.head()

,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowAvailable,BuyItNowPrice,Charity,ConvertedBuyItNowPrice,ConvertedCurrentPrice,Country,CurrentPrice,DiscountPriceInfo,EligibleForPickupDropOff,EndTime,ExcludeShipToLocation,GalleryURL,GlobalShipping,HandlingTime,HighBidder,HitCount,IntegratedMerchantCreditCardEnabled,ItemID,ListingStatus,ListingType,Location,MinimumToBid,NewBestOffer,PaymentAllowedSite,PaymentMethods,PictureURL,PostalCode,PrimaryCategoryID,PrimaryCategoryIDPath,PrimaryCategoryName,ProductID,Quantity,QuantityAvailableHint,QuantitySold,QuantitySoldByPickupInStore,QuantityThreshold,ReserveMet,ReturnPolicy,SKU,Seller,ShipToLocations,Site,StartTime,Storefront,Subtitle,TimeLeft,Title,TopRatedListing,ViewItemURLForNaturalSearch
0,False,NaN,False,22,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 102.5}",US,"{u'CurrencyID': u'USD', u'Value': 102.5}",NaN,NaN,2015-12-02T05:13:42.000Z,"[IQ, JO, YE, AL, AD, AT, BY, BA, BG, HR, CY, C...",http://thumbs2.ebaystatic.com/pict/38147604224...,False,0,"{u'FeedbackRatingStar': u'Turquoise', u'Feedba...",122,False,381476042249,Completed,Chinese,"St. George, Utah","{u'CurrencyID': u'USD', u'Value': 105.0}",False,[],[PayPal],[http://i.ebayimg.com/00/s/OTAwWDE2MDA=/z/fhsA...,84790,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'170097479'}",1,NaN,1,0,NaN,NaN,"{u'Refund': u'Money Back', u'ReturnsWithin': u...",NaN,"{u'FeedbackRatingStar': u'PurpleShooting', u'U...","[US, CA, GB, DE, AU]",US,2015-11-25T05:13:42.000Z,{u'StoreURL': u'http://stores.ebay.com/id=3180...,NaN,PT0S,Verizon Motorola Droid ULTRA 16GB Smartphone X...,NaN,http://www.ebay.com/itm/Verizon-Motorola-Droid...
1,False,NaN,False,15,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 550.0}",US,"{u'CurrencyID': u'USD', u'Value': 550.0}",NaN,NaN,2015-12-02T05:13:33.000Z,NaN,http://thumbs1.ebaystatic.com/pict/33171706517...,True,2,"{u'FeedbackRatingStar': u'Purple', u'FeedbackP...",253,False,331717065176,Completed,Chinese,"Montgomery Village, Maryland","{u'CurrencyID': u'USD', u'Value': 560.0}",False,[],[PayPal],[http://i.ebayimg.com/00/s/MTM2NVgxMDI0/z/U9YA...,20886,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'203706854'}",1,NaN,1,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,"{u'FeedbackRatingStar': u'Blue', u'UserID': u'...","[US, CA, GB, DK, RO, SK, BG, CZ, FI, HU, LV, L...",US,2015-11-25T05:13:33.000Z,NaN,NaN,PT0S,Apple iPhone 6 Plus - 64GB - Gold (Unlocked) S...,NaN,http://www.ebay.com/itm/Apple-iPhone-6-Plus-64...
2,True,NaN,True,0,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 25.0}",US,"{u'CurrencyID': u'USD', u'Value': 25.0}",NaN,NaN,2015-12-02T05:13:32.000Z,NaN,http://thumbs4.ebaystatic.com/pict/22195693439...,True,1,"{u'FeedbackRatingStar': u'None', u'FeedbackPri...",5,False,221956934395,Completed,FixedPriceItem,"Plympton, Massachusetts",NaN,False,[],[PayPal],[http://i.ebayimg.com/00/s/ODMxWDk3Ng==/z/CLQA...,02367,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'175818253'}",1,NaN,1,0,NaN,NaN,"{u'Refund': u'Money Back', u'ReturnsWithin': u...",NaN,"{u'FeedbackRatingStar': u'Turquoise', u'UserID...","[US, CA, GB, CN, MX, DE, JP, FR, AU, RU, DK, R...",US,2015-12-02T01:24:12.000Z,NaN,NaN,PT0S,Kyocera Hydro Life C6522N 5GB Black MetroPCS S...,NaN,http://www.ebay.com/itm/Kyocera-Hydro-Life-C65...
3,False,NaN,False,0,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 25.0}",US,"{u'CurrencyID': u'USD', u'Value': 25.0}",NaN,NaN,2015-12-02T05:13:20.000Z,NaN,http://thumbs1.ebaystatic.com/pict/32191955461...,True,2,NaN,121,False,321919554612,Completed,FixedPriceItem,"Niles, Michigan",NaN,False,[],[PayPal],[http://i.ebayimg.com/00/s/MTEwMVgxNDY4/z/~BQA...,49120,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,NaN,4,NaN,4,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,"{u'FeedbackRatingStar': u'Yellow

**NEXT, we will delete columns that we just don't need to do our analysis. This could even come at the end once we figure out which aren't useful, but must remember to put it here.**


## Now we will do some Feature engineering

This is to figure out which variables actually make a distinction on profitability

In [6]:
df.shape

(20000, 53)

In [7]:
del df['Charity'], df['Country'], df['ConvertedCurrentPrice'], df['DiscountPriceInfo']
del df['ListingStatus'], df['PaymentAllowedSite'], df['ReserveMet'], df['SKU'], df['Site'], df['Storefront']
del df['Subtitle'], df['TimeLeft'], df['ViewItemURLForNaturalSearch']

In [9]:
df.columns

Index([                            u'AutoPay',           u'AvailableForPickupDropOff',                    u'BestOfferEnabled',                            u'BidCount',                   u'BuyItNowAvailable',                       u'BuyItNowPrice',              u'ConvertedBuyItNowPrice',                        u'CurrentPrice',            u'EligibleForPickupDropOff',                             u'EndTime',               u'ExcludeShipToLocation',                          u'GalleryURL',
                            u'GlobalShipping',                        u'HandlingTime',                          u'HighBidder',                            u'HitCount', u'IntegratedMerchantCreditCardEnabled',                              u'ItemID',                         u'ListingType',                            u'Location',                        u'MinimumToBid',                        u'NewBestOffer',                      u'PaymentMethods',                          u'PictureURL',
                          

In [10]:
dftouse = df.copy()

**We can now make some separator variables to make listings unique**

Certain parameters that are zero are can be significant indicators of specific characteristics of an auction. For example, if the HitCount is zero, that means nobody really bothered to interact with the auction at all.